# Scraping de los datos

En este cuaderno nos encargaremos del web scraping para la obtención de datos del primer modelo de predicciones de precios de coche

## Instalación de librerías

Las librerias que necesitaremos son:

* requests
* beautifulsoup4

requests hará las peticiones y extraera la web mientras que beautifulsoup4 se encargará del scraping buscando elementos especificos en la web extraida.

In [1]:
!pip install requests beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importación de las librerías

* requests: Para peticciones http
* beautifulsoup: Para scraping
* numpy: Para transformaciones de array numericos
* re: Para extraer patrones especificos de textos en la web
* pandas: Para guardar los datos extraidos

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import pandas as pd

## Abrir acceso al drive

Lo necesitaremos para guardar nuestros datos en un csv que a su vez estará guardado en el drive y asi evitaremos que se pierda el archivo

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Declaración de variables

Declararemos las siguientes variables:

* URL: Guardaremos el link de la pagina que vamos a extraer
* header: Las cabeceras necesarias para que nuestra araña pueda scrapear sin problemas la web
* marcas: Aqui tendremos las marcas de coches que iremos extrayendo
* provincias: Tambien necesitaremos filtrar por provincias y asi tener resultados mas precisos.

In [4]:
URL = "https://www.autocasion.com/coches-ocasion?page="
hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 OPR/94.0.0.0',
       "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
       "Connection": "keep-alive",
       "Content-Encoding": "gzip",
       "Server-Timing": "matchRoutes;dur=3.3495839999999997,contextCreation;dur=2.152044,getInitialProps;dur=243.662208,renderToString;dur=18.667548999999998",
       "Vary": "Accept-Encoding",
       "Strict-Transport-Security": "max-age=31536000; includeSubDomains",}

marcas = ["abarth", "ac", "alfa-romeo", "alpine", "aston-martin", "audi", "bentley", "bmw", "cadillac", "chevrolet",
          "chrysler", "citroen", "cupra", "dacia", "daewoo", "dfsk", "dodge", "ds", "ferrari", "fiat", "fisker", "ford",
          "galloper", "honda", "hummer", "hyundai", "infiniti", "jaguar", "jeep", "kia", "lamborghini", "lancia", "land-rover",
          "lexus", "lynk-co", "lotus", "maserati", "maybach", "mazda", "mclaren", "mercedes-benz", "mg", "mini", "mitsubishi",
          "morgan", "nissan", "opel", "peugeot", "plymouth", "porsche", "renault", "rolls-royce", "rover", "saab", "seat", "skoda", "smart", "ssangyong",
          "subaru", "suzuki", "swm", "tesla", "toyota", "volkswagen", "volvo", "wiesmann"]
provincias = ["alava", "albacete", "alicante", "almeria", "asturias", "avila", "badajoz", "barcelona", "burgos", "caceres", "cadiz", "cantabria",
              "castellon", "ceuta", "ciudad-real", "cordoba", "cuenca", "girona", "granada", "guadalajara", "guipuzcoa", "huelva", "huesca", "islas-baleares",
              "jaen", "la-coruna", "la-rioja", "las-palmas", "leon", "lleida", "lugo", "madrid", "malaga", "melilla", "murcia", "navarra", "orense", "palencia",
              "pontevedra", "salamanca", "segovia", "sevilla", "soria", "sta-c-de-tenerife", "tarragona", "teruel", "toledo", "valencia", "valladolid", "vizcaya",
              "zamora", "zaragoza"]

## Creación del dataframe

Aqui crearemos la estructura de datos para nuestros datos que luego pasarán a ser analizados.

In [5]:
columns = ['localidad', 'marca', 'modelo', 'año', 'motor (CV)', 'motor (KW)', 'kilometros',
                                 'combustible', 'puertas', 'cambio', 'emisiones', 'color', 'garantía', 'vendedor', 
                                 'precio', 'maletero', 'longitud', 'altura', 'anchura', 'plazas', 'deposito', 'peso max',
                                 'carroceria', 'vel. maxima', 'c. mixto', 'c. urbano', 'extraurbano', '0-100 km/h',
                                 'cilindrada', 'cilindros', 'transmisión', 'par maximo', 'marchas', 'traccion']
car_data = pd.DataFrame(columns=columns)

## Scraping

Aqui nos pondremos a realizar el scraping y ir extrayendo datos y guardarlos

In [ ]:
for marca in marcas:
  URL = f"https://www.autocasion.com/coches-segunda-mano/{marca}-ocasion"
  print(marca)
  for provincia in provincias:
    temp_url = URL + f"/{provincia}"
    print(provincia)
    for i in range(1, 625):
      req = requests.get(temp_url + "?page=" + str(i))
      print(temp_url + "?page=" + str(i))
      page = req.content
      soup = BeautifulSoup(page, "html.parser")
      result = soup.find_all("article", class_="anuncio")
      if len(result) == 0:
        break
      for tag in result:
        carurl = tag.findChild("a" , recursive=False)["href"]
        url = "https://www.autocasion.com" + carurl
        reqcar = requests.get(url)
        pagecar = reqcar.content
        soupcar = BeautifulSoup(pagecar, "html.parser")
        result2 = soupcar.find("p", class_="direccion")
        try:
          localidad = result2.findChild("span").text[1:-1]
          marca = soupcar.find("ul", class_="breadcrumb").select("li")[1].findChild("span").text
          modelo = marca + " " + soupcar.find("ul", class_="breadcrumb").select("li")[3].findChild("span").text
          año = soupcar.find("div", class_="titulo-ficha").select("span")[1].text
          kilometros = soupcar.find("div", class_="titulo-ficha").select("span")[3].text[:-3].replace(".","")
          cambio = soupcar.find("ul", class_="datos-basicos-ficha").select("li")[4].select("span")[0].text
          color = soupcar.find("ul", class_="datos-basicos-ficha").select("li")[7].select("span")[0].text.replace("\n", "").strip()
          garantia_extraida = soupcar.find("ul", class_="datos-basicos-ficha").select("li")[6].select("span")[0].text[0:-6]
          garantia = "No" if garantia_extraida == "0" else "Sí"
          vendedor = soupcar.find("div", class_="datos-concesionario").select("p")[0].text[9:]
          precio = soupcar.find("div", class_="precio").select("span")[0].text.strip()[0:-2].replace(".","")
          cv = soupcar.find("ul", class_="tab-spec-4").select("li")[1].text[13:]
          combustible = soupcar.find("ul", class_="tab-spec-2").select("li")[0].text[11:]
          n_puertas = soupcar.find("ul", class_="tab-spec-1").select("li")[5].text[17:]
          emision_extraida = soupcar.find("ul", class_="tab-spec-5").select("li")[3].text[10:]
          emisiones = "0" if emision_extraida == "-" else emision_extraida
          maletero = max(np.int_(soupcar.find("ul", class_="tab-spec-1").select("li")[2].text[26:].replace(".", "").split("/")))
          longitud = float(soupcar.find("ul", class_="tab-spec-1").select("li")[0].select("span")[0].text[6:-3].replace(".", "")) / 10
          altura = float(soupcar.find("ul", class_="tab-spec-1").select("li")[0].select("span")[2].text[5:-3].replace(".", "")) / 10
          anchura = float(soupcar.find("ul", class_="tab-spec-1").select("li")[0].select("span")[1].text[6:-3].replace(".", "")) / 10
          plazas = soupcar.find("ul", class_="tab-spec-1").select("li")[-1].text[16:]
          deposito = soupcar.find("ul", class_="tab-spec-5").select("li")[-1].text[27:]
          peso_max = soupcar.find("ul", class_="tab-spec-1").select("li")[3].text[30:]
          carroceria = soupcar.find("ul", class_="tab-spec-1").select("li")[4].text[10:]
          velocidad_maxima_extraida = soupcar.find("ul", class_="tab-spec-4").select("li")[3].text[23:]
          velocidad_maxima = "0" if velocidad_maxima_extraida == "-" else velocidad_maxima_extraida
          c_mixto = soupcar.find("ul", class_="tab-spec-5").select("li")[2].text[16:]
          c_urbano = soupcar.find("ul", class_="tab-spec-5").select("li")[0].text[17:]
          extraurbano = soupcar.find("ul", class_="tab-spec-5").select("li")[1].text[20:]
          aceleracion = soupcar.find("ul", class_="tab-spec-4").select("li")[-1].text[26:]
          engine_kw = soupcar.find("ul", class_="tab-spec-4").select("li")[0].text[13:]
          cilindrada = soupcar.find("ul", class_="tab-spec-2").select("li")[1].text[16:].replace(".","")
          cilindros_extraido = soupcar.find("ul", class_="tab-spec-2").select("li")[2].text[19:]
          cilindros = "0" if cilindros_extraido == "-" else cilindros_extraido
          transmision = soupcar.find("ul", class_="tab-spec-3").select("li")[1].text[19:]
          par_maximo = soupcar.find("ul", class_="tab-spec-4").select("li")[2].text[8:]
          marchas = soupcar.find("ul", class_="tab-spec-3").select("li")[2].text[17:]
          traccion = soupcar.find("ul", class_="tab-spec-3").select("li")[0].text[8:]
        except:
          continue
        datos_coche = [localidad, marca, modelo, año, cv, engine_kw, kilometros, combustible, n_puertas, cambio, emisiones, color, garantia,
                        vendedor, precio, maletero, longitud, altura, anchura, plazas, deposito, peso_max, carroceria, velocidad_maxima,
                        c_mixto, c_urbano, extraurbano, aceleracion, cilindrada, cilindros, transmision, par_maximo, marchas, traccion]
        car_data.loc[len(car_data)] = datos_coche

car_data

## Randomizar los datos

Será necesario randomizar los datos para que en el entrenamiento se consiga un mejor resultado debido a que tendrá diferentes tipos de datos

In [ ]:
car_data = car_data.sample(frac=1)

## Guardado de datos

Aqui acabaremos ya guardando el dataframe en un csv que luego se guarda en el drive y lo tendremos listo para que hagamos analisis y entrenamiento con esos datos

In [ ]:
car_data.to_csv("/content/drive/MyDrive/Marcas_coches/car_data.csv")